## Classes in Python

A class can be considered a template for a new object. It has already been mentioned that many of the types in Python that have been using are really objects, meaning they can have associated methods and hold data. For example a string is an object that holds some characters and a number of helpful methods, for example upper that returns the string in capital letters. When creating a new class, a new type of object can be made from that class.

When using classes, one use object-oriented programming, and this allows the programmer to organize the code in a differnt way than you might be used to. Classes are made to correspond to different parts of a system, and then these can be assembled to more complex systems, while hiding the internal logic of the parts. This is how most modern software is structured, and will be standard for ESS software.

In this notebook the concept of classes will be illustrated by building a simulation of a rocket. We will make the following classes:

- RocketEnigne
- FuelTank
- RocketStage (consits of one engine and one fuel tank)
- Rocket (consits of a number of RocketStages)

### The rocket engine class
First we need to describe a rocket engine. The engine will be used by the RocketStage, and needs no actual functionality, but needs to hold its specifications. The `__init__` method is used whenever a new _instance_ of a class is made, so here we use the `__init__` method to store the information inside the object. Inside an object, one refers to the object itself as _self_, so the information given in `__init__` is internalized by adding it to self. Parameters added to _self_ is called attributes, and if they are added in `__init__`, they are called instance attributes. 

In [ ]:
class RocketEngine:
    """Description of a rocket engine with given thrust, fuel consuption and mass
    
    Keyword arguments:
    thrust -- thrust of the engine in N
    fuel_consuption -- fuel consumption at full thrust in kg/s
    mass -- mass of the engine in kg
    """
    def __init__(self, thrust=3500, fuel_consumption=1.0, mass=2.0):
        self.thrust = thrust # Assign the thrust given in input to self, making it a instance attribute
        self.fuel_consumption = fuel_consumption # Same procedure for fuel consumption and mass
        self.mass = mass

Now that we have a simple class, it can be used to make an engine object! The attributes can be accessed directly using `.`. 

In [ ]:
test_engine = RocketEngine(thrust=1000, fuel_consumption=1.0, mass=5.0)
print(test_engine.mass)

print(test_engine)

Using the built in print function on our RocketEngine object does not return very useful information. It is possible to customize what objects of your class should print by defining a `__repr__` method that returns a string to be printed. We will expand our RocketEngine class to include such a method.

In [ ]:
class RocketEngine:
    """Description of a rocket engine with given thrust, fuel consumption and mass
    
    Keyword arguments:
    thrust -- thrust of the engine [N]
    fuel_consumption -- fuel consumption at full thrust [kg/s]
    mass -- mass of the engine [kg]
    """
    def __init__(self, thrust=3500, fuel_consumption=1.0, mass=2.0):
        self.thrust = thrust # Assign the thrust given in input to self, making it a instance attribute
        self.fuel_consumption = fuel_consumption # Same procedure for fuel consumption and mass
        self.mass = mass
        
    def __repr__(self):
        # __repr__ is a method that will be called when using print
        output = "Rocket engine with: \n" # \n adds a new line
        output += " thrust: " + str(self.thrust) + " N \n" # it is possible to access attributes from all methods
        output += " mass: " + str(self.mass) + " kg \n"
        output += " fuel consumption: " + str(self.fuel_consumption) + " kg/s \n"
        return output

In [ ]:
test_engine = RocketEngine(thrust=1000, fuel_consumption=1, mass=5)
print(test_engine)

### The fuel tank class 
Next we consider a fuel tank. We disregard that different types of fuel typically need to be mixed, and just assign a total fuel mass along with a dry mass of the tank. Again we will add a `__repr__` method to help print the objects of this class nicely.<br/>
When using a fuel tank, it is important to know how much is left, so we add a method that will return this information to the user.<br/>
Since the mass of the fuel tank will change when fuel is consumed, we also provide a method to calculate the current mass of the fuel tank, called `get_mass`.<br/>
The last method provided is called `request_fuel`, and this will take an amount of fuel out of the tank. If the amount requested is larger than what remains, only the remaining fuel is returned.

In [ ]:
class FuelTank:
    """Description of a fuel tank with given dry mass and fuel mass
    
    Keyword arguments:
    dry_mass -- mass of tank without fuel [kg]
    fuel_mass -- initial mass of fuel [kg]
    
    Methods:
    get_fuel -- returns the amount of fuel left [kg]
    get_mass -- returns the current mass of the fuel tank [kg]
    request_fuel(amount) -- uses given amount [kg] of fuel from tank
    """   
    
    def __init__(self, dry_mass=20, fuel_mass=150):
        self.dry_mass = dry_mass
        self.fuel_capacity = fuel_mass
        self.fuel_mass = fuel_mass # will decrease during simulation
        
    def __repr__(self):
        output = "Fuel tank with: \n"
        output += " dry_mass: " + str(self.dry_mass) + " kg \n"
        output += " fuel capacity: " + str(self.fuel_capacity) + " kg \n"
        output += " fuel remainig: " + str(self.fuel_mass) + " kg \n"
        
        return output        
        
    def get_fuel(self):
        return self.fuel_mass
    
    def get_mass(self):
        return self.fuel_mass + self.dry_mass
    
    def request_fuel(self, amount):
        """returns the amount of fuel used, in case the full amount was not available
        
        amount -- the amount of fuel requested [kg]
        """
        if amount < self.fuel_mass:
            self.fuel_mass -= amount
            return amount
        else:
            used_fuel = self.fuel_mass
            self.fuel_mass = 0
            return used_fuel

Now fuel tank instances can be made using our FuelTank class. Lets check that the `get_mass`method works as expected. 

In [ ]:
test_tank = FuelTank(5, 20)
print(test_tank)
print("mass of fuel tank: ", test_tank.get_mass())

The code below demonstrates the request_fuel, run the cell multiple times with ctrl+enter to see the fuel being drained. To reset the fuel tank, run the above cell to create a new instance of FuelTank that overwrites the old empty instance.

In [ ]:
fuel_used = test_tank.request_fuel(3)
print("After taking some fuel:")
print(test_tank)
print("mass of fuel tank: ", test_tank.get_mass())
print("fuel used: ", fuel_used)

### The rocket stage class
It was chosen to make the rocket engine and fuel tank independent, but they are intended to work together. Here a stage of a rocket is described, which consists of an engine and a fuel tank. In this class we can take their interaction into account.<br/>
We once again make the `get_mass` and `get_fuel` methods so that when using a rocket stage, one does not need to understand its inner logic. <br/>
Now it is also relevant to allow for firing the engine for a given amount of time (a small time step in a simulation), and returning the thrust achieved. If the tank runs out of fuel in the timespan, the thrust is lowered to provide an average constant thrust in that (small) timespan, but in normal circumstances the constant thrust of the engine is returned.

In [ ]:
class RocketStage:
    """Description of a rocket stage with fuel tank and engine
    
    Keyword arguments:
    fuel_tank -- FuelTank object
    engine -- RocketEngine object
    
    Methods:
    get_fuel -- returns the amount of fuel left
    get_mass -- returns the current mass of the fuel tank
    fire_engine(time) -- return average thrust when engine fired for time seconds
    """    
    def __init__(self, fuel_tank, engine):
        self.fuel_tank = fuel_tank
        self.engine = engine
        
    def __repr__(self):
        output = "Rocket stage that contains: \n"
        output += self.fuel_tank.__repr__() # use __repr__ method defined in FuelTank
        output += self.engine.__repr__() # use __repr__ method defined in RocketEngine
        return output
        
    def get_fuel(self):
        return self.fuel_tank.get_fuel()
        
    def get_mass(self):
        return self.fuel_tank.get_mass() + self.engine.mass
    
    def fire_engine(self, time):
        fuel_needed = time*self.engine.fuel_consumption
        fuel_used = self.fuel_tank.request_fuel(fuel_needed)
        
        return self.engine.thrust*fuel_used/fuel_needed

We can test this new class by using our `test_tank` and `test_engine` from earlier:

In [ ]:
test_stage = RocketStage(test_tank, test_engine)
print(test_stage)

It is also possible to create the necessary objects when initializing the RocketStage.

In [ ]:
test_stage = RocketStage(FuelTank(5, 20), RocketEngine(1000, 1, 5))

In [ ]:
# Use ctrl+enter to fire the engine and see the fuel being used
test_stage.fire_engine(1.2)
print(test_stage)

### The rocket class
Now that we have a class describing a rocket stage, it is much simpler to build a class that describes a rocket of several stages and includes the physics for a simple simulation of a launch.

The initialize method of the rocket takes a list of stages, assuming the first stage is the one to be expended first. A few additional constants are provided, such as the payload mass, simulation time step, maximum simulation duration and gravity. The `__init__` method also sets initial values for time, elevation and speed used in the simulation, as well setting up lists for recording data under the simulation.

The following methods are then created:
- get_mass: Calculates the mass of the entire rocket, as it changes when stages are removed and fuel is spent
- eject_stage: Will eject the current stage
- record_data(acceleration): Will record the current state of the rocket to the data lists, including given acceleration.
- update_trjajectory(acceleration): Uses classical mechanics to update position and speed from acceleration
- simulate_launch: Method that contains the logic for simulating a launch, including calculation of acceleration
- A `__repr__` method is missing, the next task will be to add this feature

In [ ]:
class Rocket:
    """Description of a rocket with a number of stages and payload
    
    Keyword arguments:
    stages -- List of RocketStage objects
    payload_mass -- Mass of payload [kg]
    time_step -- Time step used in simulation in [s]
    gravity -- Gravity acceleration in [m/s^2]
    
    Methods:
    get_mass -- returns the current mass of the rocket [kg]
    eject_stage -- ejects the lowest stage, index 0 in the stages list
    record_data(acceleration) -- records current rocket state including acceleration to data
    update_trajectory(acceleration) -- updates the state of the rocket using clasical mechanics
    simulate_launch -- performs a simulation of the rocket being launched
    """      
    def __init__(self, stages, payload_mass, time_step=0.1, max_time=1000, gravity=-9.80665):
        # Transfer input to instance attributes
        self.stages = stages # list of stages, stage 0 is first to burn
        self.payload_mass = payload_mass
        self.time_step = time_step
        self.max_time = max_time
        self.gravity = gravity

        # Initial state of the rocket
        self.time = 0
        self.elevation = 0
        self.speed = 0

        # Initialize lists for holding data points
        self.time_data = []
        self.mass_data = []
        self.elevation_data = []
        self.speed_data = []
        self.acceleration_data = []
    
    # -- YOUR CODE HERE --
    # ---------------------
    
    def get_mass(self):
        mass = self.payload_mass
        for stage in self.stages:
            mass += stage.get_mass()
            
        return mass
            
    def eject_stage(self):
        self.stages.pop(0) # removes the lowest stage from the rocket!
        
    def record_data(self, acceleration):
        self.time_data.append(self.time)
        self.mass_data.append(self.get_mass())
        self.elevation_data.append(self.elevation)
        self.speed_data.append(self.speed)
        self.acceleration_data.append(acceleration + self.gravity)
        
    def update_trajectory(self, acceleration):
        self.time += self.time_step
        self.speed += acceleration*self.time_step + self.gravity*self.time_step
        self.elevation += self.speed*self.time_step
        
    def simulate_launch(self):
        while (len(self.stages) > 0 and self.time < self.max_time):
            # Calculate thrust and mass
            thrust = self.stages[0].fire_engine(self.time_step)
            mass = self.get_mass()
            
            # Get acceleration from Newtons 2nd law
            acceleration = thrust/mass 

            # Update the rocket trajectory and record the state
            self.update_trajectory(acceleration)
            self.record_data(acceleration)
            
            # Check if the current stage is empty, and eject it if so
            if self.stages[0].get_fuel() == 0:
                self.eject_stage()

**(1) Task:** <br>
The rocket class lacks a `__repr__` method for printing its state, add such a method to the above code.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
stage1_fuel = FuelTank(dry_mass=15, fuel_mass=50)
stage1_engine = RocketEngine(thrust=1600, fuel_consumption=4, mass=3)
stage1 = RocketStage(stage1_fuel, stage1_engine)

stage2_fuel = FuelTank(dry_mass=2, fuel_mass=15)
stage2_engine = RocketEngine(thrust=1200, fuel_consumption=2, mass=1)
stage2 = RocketStage(stage2_fuel, stage2_engine)

model_rocket = Rocket([stage1, stage2], payload_mass=1.5, time_step=0.10)

print(model_rocket)

model_rocket.simulate_launch()

plt.plot(model_rocket.time_data, model_rocket.elevation_data, '-')
plt.xlabel("time [s]")
plt.ylabel("elevation [m]")
plt.show()

plt.plot(model_rocket.time_data, model_rocket.speed_data, '-')
plt.xlabel("time [s]")
plt.ylabel("speed [m/s]")
plt.show()

plt.plot(model_rocket.time_data, model_rocket.acceleration_data, '-')
plt.xlabel("time [s]")
plt.ylabel("acceleration [m/s/s]")
plt.show()

plt.plot(model_rocket.time_data, model_rocket.mass_data, '-')
plt.xlabel("time [s]")
plt.ylabel("rocket mass [kg]")
plt.show()

**(2) Task:** <br>
The simulation uses a finite timestep, investigate how this affects the simulation by comparing two otherwise identical simulations.